<a href="https://colab.research.google.com/github/Suesue098/Basic/blob/main/Tank_Management_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sys # Used for exiting gracefully
from urllib.parse import urljoin # To construct absolute URLs from relative paths
import warnings # To suppress SSL warnings
import time

warnings.filterwarnings("ignore") #supress all warnings

# --- Configuration ---
# TODO: Replace these placeholders with actual values from the website
LOGIN_URL = 'https://pyrataquatics.janelia.org/aquatic/frontend/session/login'  # The URL the login form POSTs to
CSV_FILENAME = 'output_table.csv' # Name for the output CSV file

# --- Credentials ---
# WARNING: Avoid hardcoding credentials in production code.
# Consider environment variables or other secure methods.
# --- Credentials ---
USERNAME = 'yans2'
PASSWORD = 'Jtwmy&Dtsgx2025'
# --- Login Form Field Names ---
# TODO: Find these using browser developer tools on the login page
USERNAME_FIELD_NAME = 'username' # The 'name' attribute of the username input field
PASSWORD_FIELD_NAME = 'password' # The 'name' attribute of the password input field
# Add any other required hidden form fields you find by inspecting the login form
# OTHER_LOGIN_FORM_FIELDS = {'csrf_token': 'fetch_this_value', 'login_action': 'submit'}

# --- Choose Alias Form Configuration ---
# TODO: Set the desired value for the 'user_id' dropdown in the "Choose alias" form.
#       Find the correct value attribute from the <option> tags in the HTML.
#       Defaulting to "" which corresponds to "No alias" in the example HTML.
CHOOSE_ALIAS_USER_ID = "" # e.g., "7" for "Ahrens Misha", "" for "No alias"

# --- Table Identifier ---
# TODO: Identify the table you want on the *final* page after submitting the alias form.
# 1. If the table has a unique ID: TABLE_ID = 'your-table-id'
# 2. If not, you might need to select by order (e.g., the first table is index 0)
TABLE_ID = None # Set this if the table has an ID, e.g., 'dataTable'
TABLE_INDEX = 0 # If selecting by order, which table is it? (0 for first, 1 for second, etc.)

# --- Pagination Configuration ---
# Optional delay between fetching pages to avoid overwhelming the server
PAGE_FETCH_DELAY_SECONDS = 1 # Set to 0 for no delay
# --- SSL Verification ---
# Set to False to disable SSL certificate verification (USE WITH CAUTION)
# Set to True (default) or a path to a CA bundle file to enable verification.
VERIFY_SSL = False

# Suppress only the single InsecureRequestWarning from urllib3 needed for `verify=False`
if not VERIFY_SSL:
    warnings.filterwarnings("ignore", message="Unverified HTTPS request")


In [54]:
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': LOGIN_URL
})

all_dataframes = [] # List to store DataFrames from each page
current_page_num = 1
total_pages = None # Will try to detect this

try:
    # 1. Initial Login
    print(f"Logging in...")
    login_payload = {
        USERNAME_FIELD_NAME: USERNAME,
        PASSWORD_FIELD_NAME: PASSWORD,
        # **OTHER_LOGIN_FORM_FIELDS
    }
    #print("Sending login POST request...")
    login_response = session.post(LOGIN_URL, data=login_payload, verify=VERIFY_SSL)
    login_response.raise_for_status()
    intermediate_page_url = login_response.url
    print(f"Login successful.")


    # 2. Handle the Intermediate "Choose Alias" Form
    #print("Processing intermediate 'Choose Alias' page...")
    intermediate_page_content = login_response.text
    intermediate_page_soup = BeautifulSoup(intermediate_page_content, 'lxml')
    alias_form = intermediate_page_soup.find('form', {'action': '/aquatic/frontend/session/choose_alias'})
    if not alias_form:
        print("Error: Could not find the 'Choose Alias' form.")
        sys.exit(1)
    sessionid_input = alias_form.find('input', {'name': 'sessionid', 'type': 'hidden'})
    if not sessionid_input or 'value' not in sessionid_input.attrs:
        print("Error: Could not find the hidden 'sessionid' input.")
        sys.exit(1)
    sessionid_value = sessionid_input['value']
    relative_action_url = alias_form.get('action')
    if not relative_action_url:
         print("Error: Could not find the action attribute on the alias form.")
         sys.exit(1)
    alias_submit_url = urljoin(intermediate_page_url, relative_action_url)
    alias_payload = {'sessionid': sessionid_value, 'user_id': CHOOSE_ALIAS_USER_ID}
    #print(f"Submitting 'Choose Alias' form to {alias_submit_url}...")
    session.headers.update({'Referer': intermediate_page_url})
    current_page_response = session.post(alias_submit_url, data=alias_payload, verify=VERIFY_SSL)
    current_page_response.raise_for_status()
    current_page_url = current_page_response.url
    print(f"Landed on first data page")

    # 3. Process Pages and Extract Tables
    while True: # Loop through pages
        print(f"Processing page {current_page_num}...")
        current_page_content = current_page_response.text
        current_page_soup = BeautifulSoup(current_page_content, 'lxml')

        # --- Extract Total Pages (for logging, done on first page or if not yet found) ---
        if total_pages is None:
            try:
                paginator = current_page_soup.find('div', class_='paginator')
                if paginator:
                    num_pages_span = paginator.find('span', class_='num-pages')
                    if num_pages_span and num_pages_span.b:
                        total_pages = int(num_pages_span.b.get_text(strip=True))
                        print(f"Detected {total_pages} total pages.")
            except Exception as e:
                print(f"Could not detect total pages (optional): {e}")
        # -----------------------------------------------------------------------------

        # --- Extract Table from Current Page ---
        try:
            if TABLE_ID:
                tables = pd.read_html(current_page_content, attrs={'id': TABLE_ID}, flavor='lxml')
            else:
                tables = pd.read_html(current_page_content, flavor='lxml')

            if not tables:
                print(f"Warning: No tables found on page {current_page_num}.")
                # Decide if this is an error or expected (e.g., last page might be empty)
                # If it's an error, you might want to 'break' or 'sys.exit(1)'
            elif TABLE_ID and len(tables) >= 1:
                 #print(f"Extracted table with ID '{TABLE_ID}' from page {current_page_num}.")
                 all_dataframes.append(tables[0])
            elif not TABLE_ID and len(tables) > TABLE_INDEX:
                 #print(f"Extracted table at index {TABLE_INDEX} from page {current_page_num}.")
                 all_dataframes.append(tables[TABLE_INDEX])
            else:
                 print(f"Warning: Could not find the specified table (ID='{TABLE_ID}', Index={TABLE_INDEX}) on page {current_page_num}, though other tables might exist.")

        except ValueError as e:
            # This can happen if pd.read_html finds no <table> tags at all
            print(f"Error parsing tables with pandas on page {current_page_num}: {e}")
            # Decide how to handle: continue, break, or exit
            # break # Example: stop processing if a page fails to parse
        except Exception as e:
            print(f"An unexpected error occurred parsing table on page {current_page_num}: {e}")
            # break # Example: stop processing on unexpected errors
        # -----------------------------------------

        # --- Find Next Page Link ---
        paginator = current_page_soup.find('div', class_='paginator')
        next_page_link = None
        if paginator:
            # Find the 'a' tag for the single right arrow that does NOT have the 'disabled' class
            next_page_link = paginator.find('a', class_=lambda c: c and 'icon_single_right_arrow_bold' in c.split() and 'disabled' not in c.split())

        if next_page_link and next_page_link.get('href'):
            next_page_href = next_page_link.get('href')
            next_page_url = urljoin(current_page_url, next_page_href)
            #print(f"Found next page link: {next_page_url}")

            # Optional delay
            if PAGE_FETCH_DELAY_SECONDS > 0:
                #print(f"Waiting {PAGE_FETCH_DELAY_SECONDS}s before fetching next page...")
                time.sleep(PAGE_FETCH_DELAY_SECONDS)

            # Fetch the next page
            #print("Fetching next page...")
            session.headers.update({'Referer': current_page_url}) # Update referer
            current_page_response = session.get(next_page_url, verify=VERIFY_SSL)
            current_page_response.raise_for_status()
            current_page_url = current_page_response.url # Update current URL
            current_page_num += 1
        else:
            print("All pages processed.")
            break # Exit the loop if no next page link is found
        # -------------------------

    # 4. Combine and Save Data
    if not all_dataframes:
        print("No data was extracted from any page.")
        sys.exit(0) # Exit gracefully if no data

    #print(f"\nCombining data from {len(all_dataframes)} pages...")
    # Concatenate all collected dataframes
    combined_df = pd.concat(all_dataframes, ignore_index=True)

    print(f"Processed {len(combined_df)} records.")
    #print(f"Saving combined table to '{CSV_FILENAME}'...")
    # index=False prevents pandas from writing the DataFrame index as a column
    combined_df.to_csv(CSV_FILENAME, index=False, encoding='utf-8')
    print("Data successfully saved to drive.")

except requests.exceptions.SSLError as e:
    print(f"SSL Error occurred: {e}")
    print("This might be due to an untrusted certificate. Check VERIFY_SSL setting.")
except requests.exceptions.RequestException as e:
    print(f"An error occurred during a web request: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Response Status Code: {e.response.status_code}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    import traceback
    traceback.print_exc()

finally:
    session.close()
    print("Script finished.")


Logging in...
Login successful.
Landed on first data page
Processing page 1...
Detected 9 total pages.
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
All pages processed.
Processed 844 records.
Data successfully saved to drive.
Script finished.


In [55]:
import pandas as pd


In [56]:
df = pd.read_csv("output_table.csv")

In [57]:
df.head(5)

,Check all,Unnamed: 1,WR,Rack,Position,Tank ID,Label,Unnamed: 7,% juvenile age,% adult age,...,Parents IDs,Owner,Children IDs,Animal age (d),Last crossing,Unnamed: 20,Unnamed: 6,Unnamed: 8,Unnamed: 12,Unnamed: 21
0,Item 5822,⌁,NaN,DNLA Virtual Rack 1 Location Building: LSBAr...,1,5822,NaN,NaN,100.0,100.0,...,5602,Satou Chie,"6632, 6677, 6678, 6707",295,02/21/2025,NaN,NaN,NaN,NaN,NaN
1,Item 6543,⌁,NaN,NaN,NaN,5570/6543,NaN,NaN,76.0,76.0,...,4970,Satou Chie,"6632, 6677, 6678, 6707",376,02/21/2025,NaN,NaN,NaN,NaN,NaN
2,Item 6368,⌁,NaN,DNLA Virtual Rack 1 Location Building: LSBAr...,2,6002/6368,NaN,NaN,83.0,83.0,...,5729,Satou Chie,"6688, 6700, 6714, 6748",245,02/24/2025,NaN,NaN,NaN,NaN,NaN
3,Item 6542,⌁,NaN,NaN,NaN,5570/6542,NaN,NaN,76.0,76.0,...,4970,Satou Chie,"6688, 6700, 6714, 6748",376,02/24/2025,NaN,NaN,NaN,NaN,NaN
4,Item 6538,⌁,NaN,DNLA Virtual Rack 1 Location Building: LSBAr...,3,6538,NaN,NaN,25.0,25.0,...,6129,Satou Chie,"6737, 6761, 6816, 6825, 6871",124,04/01/2025,NaN,NaN,NaN,NaN,NaN


In [58]:
# Drop all columns whose names start with 'Unnamed' and "WR"
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Drop specific columns
df = df.drop(columns=['WR'])
df = df.drop(columns=['Tank type'])
df.head(5)

,Check all,Rack,Position,Tank ID,Label,% juvenile age,% adult age,# Total,Age level,Line / Strain (Name),Parents IDs,Owner,Children IDs,Animal age (d),Last crossing
0,Item 5822,DNLA Virtual Rack 1 Location Building: LSBAr...,1,5822,NaN,100.0,100.0,1,Adult,HuC:Nes-Halotag-EGFP [ZT012],5602,Satou Chie,"6632, 6677, 6678, 6707",295,02/21/2025
1,Item 6543,NaN,NaN,5570/6543,NaN,76.0,76.0,2,Adult,mitfa -/-,4970,Satou Chie,"6632, 6677, 6678, 6707",376,02/21/2025
2,Item 6368,DNLA Virtual Rack 1 Location Building: LSBAr...,2,6002/6368,NaN,83.0,83.0,3,Adult,UBBR:GCaMP8m [ZT013],5729,Satou Chie,"6688, 6700, 6714, 6748",245,02/24/2025
3,Item 6542,NaN,NaN,5570/6542,NaN,76.0,76.0,4,Adult,mitfa -/-,4970,Satou Chie,"6688, 6700, 6714, 6748",376,02/24/2025
4,Item 6538,DNLA Virtual Rack 1 Location Building: LSBAr...,3,6538,NaN,25.0,25.0,5,Adult,Tol2 HuC HS Akaluc cmlc2 GFP [ZT015],6129,Satou Chie,"6737, 6761, 6816, 6825, 6871",124,04/01/2025


In [59]:
# Drop rows where 'Rack' contains 'DNLA' or 'SAAB'
df = df[~df['Rack'].astype(str).str.contains('DNLA|SAAB', na=False)]
df.head(10)

,Check all,Rack,Position,Tank ID,Label,% juvenile age,% adult age,# Total,Age level,Line / Strain (Name),Parents IDs,Owner,Children IDs,Animal age (d),Last crossing
1,Item 6543,NaN,NaN,5570/6543,NaN,76.0,76.0,2,Adult,mitfa -/-,4970,Satou Chie,"6632, 6677, 6678, 6707",376,02/21/2025
3,Item 6542,NaN,NaN,5570/6542,NaN,76.0,76.0,4,Adult,mitfa -/-,4970,Satou Chie,"6688, 6700, 6714, 6748",376,02/24/2025
7,Item 6497,NaN,NaN,5569/6497,NaN,74.0,74.0,2,Adult,mitfa -/-,4970,Satou Chie,"6679, 6706, 6815, 6824",377,03/26/2025
9,Item 6757,NaN,NaN,6387/6757,NaN,84.0,84.0,3,Adult,mitfa -/-,5417,Satou Chie,6853,154,04/05/2025
10,Item 5131,M-F09 Location Building: LSBArea: Aquatics 2...,A2,4555/5131,CID 12118-1,100.0,75.0,1,Adult,Tg(PB4:tTA;cryaa:mRuby); Tg(TRE:TEMPO-gRNA1),"3377, 3148",Ilanges Anoj,"5115, 5470, 5471",659,01/22/2024
11,Item 5868,M-F09 Location Building: LSBArea: Aquatics 2...,A4,4883/5868,CID 13675-3,100.0,160.0,1,Adult,Tg(acta2:CoChR-eGFP),"3746, 3747",Ilanges Anoj,NaN,550,05/03/2025
12,Item 6157,M-F09 Location Building: LSBArea: Aquatics 2...,A5,5049/6157,CID 14197-3,100.0,56.0,1,Adult,Tg(ubi:tTA; TRE:jRGECO1b); (acta2:gtacr2-eYFP ),"4248, 4689, 4866",Ilanges Anoj,"6127, 6128",504,05/08/2025
13,Item 5299,M-F09 Location Building: LSBArea: Aquatics 2...,A9,4409/5299,CID 12359-1,60.0,60.0,1,Adult,Tg(her4.1:PMCA2-mCherry_J05),"3124, 3256",Ahrens Misha,"4907, 4908, 4909, 4953, 4967, 5208",713,02/25/2024
14,Item 6159,M-F09 Location Building: LSBArea: Aquatics 2...,A10,4903/6159,CID 14246-2 (no signal last time),42.0,42.0,1,Adult,Tg(isl1CREST-hsp70l:tTA); Tg(TRE:stGtACR2-eYFP),"3746, 3747",Ilanges Anoj,6147,544,05/09/2025
15,Item 6160,NaN,NaN,4898/6160,CID 14255-2,60.0,60.0,1,Adult,Tg(ubi:tTa; TRE:jRGECO1b); (TRE:CoChR-eGFP),"4248, 4689, 3746",Ilanges Anoj,6171,545,05/08/2025


In [60]:
df['Children IDs'] = df['Children IDs'].fillna('Null')

In [61]:
df['Children Tank'] = df['Children IDs'].apply(
    lambda x: 'No' if pd.isna(x) or str(x).strip() == 'Null' else 'Yes'
)

In [62]:
df.head(25)

,Check all,Rack,Position,Tank ID,Label,% juvenile age,% adult age,# Total,Age level,Line / Strain (Name),Parents IDs,Owner,Children IDs,Animal age (d),Last crossing,Children Tank
1,Item 6543,NaN,NaN,5570/6543,NaN,76.0,76.0,2,Adult,mitfa -/-,4970,Satou Chie,"6632, 6677, 6678, 6707",376,02/21/2025,Yes
3,Item 6542,NaN,NaN,5570/6542,NaN,76.0,76.0,4,Adult,mitfa -/-,4970,Satou Chie,"6688, 6700, 6714, 6748",376,02/24/2025,Yes
7,Item 6497,NaN,NaN,5569/6497,NaN,74.0,74.0,2,Adult,mitfa -/-,4970,Satou Chie,"6679, 6706, 6815, 6824",377,03/26/2025,Yes
9,Item 6757,NaN,NaN,6387/6757,NaN,84.0,84.0,3,Adult,mitfa -/-,5417,Satou Chie,6853,154,04/05/2025,Yes
10,Item 5131,M-F09 Location Building: LSBArea: Aquatics 2...,A2,4555/5131,CID 12118-1,100.0,75.0,1,Adult,Tg(PB4:tTA;cryaa:mRuby); Tg(TRE:TEMPO-gRNA1),"3377, 3148",Ilanges Anoj,"5115, 5470, 5471",659,01/22/2024,Yes
11,Item 5868,M-F09 Location Building: LSBArea: Aquatics 2...,A4,4883/5868,CID 13675-3,100.0,160.0,1,Adult,Tg(acta2:CoChR-eGFP),"3746, 3747",Ilanges Anoj,Null,550,05/03/2025,No
12,Item 6157,M-F09 Location Building: LSBArea: Aquatics 2...,A5,5049/6157,CID 14197-3,100.0,56.0,1,Adult,Tg(ubi:tTA; TRE:jRGECO1b); (acta2:gtacr2-eYFP ),"4248, 4689, 4866",Ilanges Anoj,"6127, 6128",504,05/08/2025,Yes
13,Item 5299,M-F09 Location Building: LSBArea: Aquatics 2...,A9,4409/5299,CID 12359-1,60.0,60.0,1,Adult,Tg(her4.1:PMCA2-mCherry_J05),"3124, 3256",Ahrens Misha,"4907, 4908, 4909, 4953, 4967, 5208",713,02/25/2024,Yes
14,Item 6159,M-F09 Location Building: LSBArea: Aquatics 2...,A10,4903/6159,CID 14246-2 (no signal last time),42.0,42.0,1,Adult,Tg(isl1CREST-hsp70l:tTA); Tg(TRE:stGtACR2-eYFP),"3746, 3747",Ilanges Anoj,6147,544,05/09/2025,Yes
15,Item 6160,NaN,NaN,4898/6160,CID 14255-2,60.0,60.0,1,Adult,Tg(ubi:tTa; TRE:jRGECO1b); (TRE:CoChR-eGFP),"4248, 4689, 3746",Ilanges Anoj,6171,545,05/08/2025,Yes


In [63]:
def check_children_adults(row):
    if row['Children Tank'] == 'No':
        return 'No'

    child_ids = eval(row['Children IDs']) if row['Children IDs'] != 'Null' else []
    for child_id in child_ids:
        if child_id in df['Tank ID'].values:
            age_lev = df.loc[df['Tank ID'] == child_id, 'Age level'].values[0]
            if age_lev == 'Adult':
                return 'Yes'
    return 'No'

In [64]:
df.head(15)

,Check all,Rack,Position,Tank ID,Label,% juvenile age,% adult age,# Total,Age level,Line / Strain (Name),Parents IDs,Owner,Children IDs,Animal age (d),Last crossing,Children Tank
1,Item 6543,NaN,NaN,5570/6543,NaN,76.0,76.0,2,Adult,mitfa -/-,4970,Satou Chie,"6632, 6677, 6678, 6707",376,02/21/2025,Yes
3,Item 6542,NaN,NaN,5570/6542,NaN,76.0,76.0,4,Adult,mitfa -/-,4970,Satou Chie,"6688, 6700, 6714, 6748",376,02/24/2025,Yes
7,Item 6497,NaN,NaN,5569/6497,NaN,74.0,74.0,2,Adult,mitfa -/-,4970,Satou Chie,"6679, 6706, 6815, 6824",377,03/26/2025,Yes
9,Item 6757,NaN,NaN,6387/6757,NaN,84.0,84.0,3,Adult,mitfa -/-,5417,Satou Chie,6853,154,04/05/2025,Yes
10,Item 5131,M-F09 Location Building: LSBArea: Aquatics 2...,A2,4555/5131,CID 12118-1,100.0,75.0,1,Adult,Tg(PB4:tTA;cryaa:mRuby); Tg(TRE:TEMPO-gRNA1),"3377, 3148",Ilanges Anoj,"5115, 5470, 5471",659,01/22/2024,Yes
11,Item 5868,M-F09 Location Building: LSBArea: Aquatics 2...,A4,4883/5868,CID 13675-3,100.0,160.0,1,Adult,Tg(acta2:CoChR-eGFP),"3746, 3747",Ilanges Anoj,Null,550,05/03/2025,No
12,Item 6157,M-F09 Location Building: LSBArea: Aquatics 2...,A5,5049/6157,CID 14197-3,100.0,56.0,1,Adult,Tg(ubi:tTA; TRE:jRGECO1b); (acta2:gtacr2-eYFP ),"4248, 4689, 4866",Ilanges Anoj,"6127, 6128",504,05/08/2025,Yes
13,Item 5299,M-F09 Location Building: LSBArea: Aquatics 2...,A9,4409/5299,CID 12359-1,60.0,60.0,1,Adult,Tg(her4.1:PMCA2-mCherry_J05),"3124, 3256",Ahrens Misha,"4907, 4908, 4909, 4953, 4967, 5208",713,02/25/2024,Yes
14,Item 6159,M-F09 Location Building: LSBArea: Aquatics 2...,A10,4903/6159,CID 14246-2 (no signal last time),42.0,42.0,1,Adult,Tg(isl1CREST-hsp70l:tTA); Tg(TRE:stGtACR2-eYFP),"3746, 3747",Ilanges Anoj,6147,544,05/09/2025,Yes
15,Item 6160,NaN,NaN,4898/6160,CID 14255-2,60.0,60.0,1,Adult,Tg(ubi:tTa; TRE:jRGECO1b); (TRE:CoChR-eGFP),"4248, 4689, 3746",Ilanges Anoj,6171,545,05/08/2025,Yes


In [65]:
def check_children_adults(row):
    if row['Children Tank'] != 'Yes':
        return 'No'

    # Safely parse the Children IDs
    if pd.isna(row['Children IDs']):
        return 'No'

    # Split the string into a list of IDs, stripping spaces
    child_ids = [id_.strip() for id_ in str(row['Children IDs']).split(',') if id_.strip().isdigit()]

    for child_id in child_ids:
        matching_rows = df[df['Tank ID'].astype(str).str.endswith(child_id)]
        if not matching_rows.empty:
            age_lev = matching_rows.iloc[0]['Age level']
            if age_lev == 'Adult':
                return 'Yes'
    return 'No'


In [66]:
df['Children Adults?'] = df.apply(check_children_adults, axis=1)

In [67]:
df.head(15)

,Check all,Rack,Position,Tank ID,Label,% juvenile age,% adult age,# Total,Age level,Line / Strain (Name),Parents IDs,Owner,Children IDs,Animal age (d),Last crossing,Children Tank,Children Adults?
1,Item 6543,NaN,NaN,5570/6543,NaN,76.0,76.0,2,Adult,mitfa -/-,4970,Satou Chie,"6632, 6677, 6678, 6707",376,02/21/2025,Yes,No
3,Item 6542,NaN,NaN,5570/6542,NaN,76.0,76.0,4,Adult,mitfa -/-,4970,Satou Chie,"6688, 6700, 6714, 6748",376,02/24/2025,Yes,No
7,Item 6497,NaN,NaN,5569/6497,NaN,74.0,74.0,2,Adult,mitfa -/-,4970,Satou Chie,"6679, 6706, 6815, 6824",377,03/26/2025,Yes,No
9,Item 6757,NaN,NaN,6387/6757,NaN,84.0,84.0,3,Adult,mitfa -/-,5417,Satou Chie,6853,154,04/05/2025,Yes,No
10,Item 5131,M-F09 Location Building: LSBArea: Aquatics 2...,A2,4555/5131,CID 12118-1,100.0,75.0,1,Adult,Tg(PB4:tTA;cryaa:mRuby); Tg(TRE:TEMPO-gRNA1),"3377, 3148",Ilanges Anoj,"5115, 5470, 5471",659,01/22/2024,Yes,Yes
11,Item 5868,M-F09 Location Building: LSBArea: Aquatics 2...,A4,4883/5868,CID 13675-3,100.0,160.0,1,Adult,Tg(acta2:CoChR-eGFP),"3746, 3747",Ilanges Anoj,Null,550,05/03/2025,No,No
12,Item 6157,M-F09 Location Building: LSBArea: Aquatics 2...,A5,5049/6157,CID 14197-3,100.0,56.0,1,Adult,Tg(ubi:tTA; TRE:jRGECO1b); (acta2:gtacr2-eYFP ),"4248, 4689, 4866",Ilanges Anoj,"6127, 6128",504,05/08/2025,Yes,Yes
13,Item 5299,M-F09 Location Building: LSBArea: Aquatics 2...,A9,4409/5299,CID 12359-1,60.0,60.0,1,Adult,Tg(her4.1:PMCA2-mCherry_J05),"3124, 3256",Ahrens Misha,"4907, 4908, 4909, 4953, 4967, 5208",713,02/25/2024,Yes,No
14,Item 6159,M-F09 Location Building: LSBArea: Aquatics 2...,A10,4903/6159,CID 14246-2 (no signal last time),42.0,42.0,1,Adult,Tg(isl1CREST-hsp70l:tTA); Tg(TRE:stGtACR2-eYFP),"3746, 3747",Ilanges Anoj,6147,544,05/09/2025,Yes,No
15,Item 6160,NaN,NaN,4898/6160,CID 14255-2,60.0,60.0,1,Adult,Tg(ubi:tTa; TRE:jRGECO1b); (TRE:CoChR-eGFP),"4248, 4689, 3746",Ilanges Anoj,6171,545,05/08/2025,Yes,Yes


In [68]:
def check_children_adults(row):
    if row['Children Tank'] != 'Yes':
        return 'No', 0

    if pd.isna(row['Children IDs']):
        return 'No', 0

    # Split string into list of IDs
    child_ids = [id_.strip() for id_ in str(row['Children IDs']).split(',') if id_.strip().isdigit()]

    adult_count = 0
    for child_id in child_ids:
        matching_rows = df[df['Tank ID'].astype(str).str.endswith(child_id)]
        if not matching_rows.empty:
            age_lev = matching_rows.iloc[0]['Age level']
            if age_lev == 'Adult':
                adult_count += matching_rows.iloc[0]['# Total']
    return ('Yes', adult_count) if adult_count > 0 else ('No', 0)


In [69]:
df[['Children Adults?', '# children adults']] = df.apply(lambda row: pd.Series(check_children_adults(row)), axis=1)

In [70]:
df.head(15)

,Check all,Rack,Position,Tank ID,Label,% juvenile age,% adult age,# Total,Age level,Line / Strain (Name),Parents IDs,Owner,Children IDs,Animal age (d),Last crossing,Children Tank,Children Adults?,# children adults
1,Item 6543,NaN,NaN,5570/6543,NaN,76.0,76.0,2,Adult,mitfa -/-,4970,Satou Chie,"6632, 6677, 6678, 6707",376,02/21/2025,Yes,No,0
3,Item 6542,NaN,NaN,5570/6542,NaN,76.0,76.0,4,Adult,mitfa -/-,4970,Satou Chie,"6688, 6700, 6714, 6748",376,02/24/2025,Yes,No,0
7,Item 6497,NaN,NaN,5569/6497,NaN,74.0,74.0,2,Adult,mitfa -/-,4970,Satou Chie,"6679, 6706, 6815, 6824",377,03/26/2025,Yes,No,0
9,Item 6757,NaN,NaN,6387/6757,NaN,84.0,84.0,3,Adult,mitfa -/-,5417,Satou Chie,6853,154,04/05/2025,Yes,No,0
10,Item 5131,M-F09 Location Building: LSBArea: Aquatics 2...,A2,4555/5131,CID 12118-1,100.0,75.0,1,Adult,Tg(PB4:tTA;cryaa:mRuby); Tg(TRE:TEMPO-gRNA1),"3377, 3148",Ilanges Anoj,"5115, 5470, 5471",659,01/22/2024,Yes,Yes,15
11,Item 5868,M-F09 Location Building: LSBArea: Aquatics 2...,A4,4883/5868,CID 13675-3,100.0,160.0,1,Adult,Tg(acta2:CoChR-eGFP),"3746, 3747",Ilanges Anoj,Null,550,05/03/2025,No,No,0
12,Item 6157,M-F09 Location Building: LSBArea: Aquatics 2...,A5,5049/6157,CID 14197-3,100.0,56.0,1,Adult,Tg(ubi:tTA; TRE:jRGECO1b); (acta2:gtacr2-eYFP ),"4248, 4689, 4866",Ilanges Anoj,"6127, 6128",504,05/08/2025,Yes,Yes,28
13,Item 5299,M-F09 Location Building: LSBArea: Aquatics 2...,A9,4409/5299,CID 12359-1,60.0,60.0,1,Adult,Tg(her4.1:PMCA2-mCherry_J05),"3124, 3256",Ahrens Misha,"4907, 4908, 4909, 4953, 4967, 5208",713,02/25/2024,Yes,No,0
14,Item 6159,M-F09 Location Building: LSBArea: Aquatics 2...,A10,4903/6159,CID 14246-2 (no signal last time),42.0,42.0,1,Adult,Tg(isl1CREST-hsp70l:tTA); Tg(TRE:stGtACR2-eYFP),"3746, 3747",Ilanges Anoj,6147,544,05/09/2025,Yes,No,0
15,Item 6160,NaN,NaN,4898/6160,CID 14255-2,60.0,60.0,1,Adult,Tg(ubi:tTa; TRE:jRGECO1b); (TRE:CoChR-eGFP),"4248, 4689, 3746",Ilanges Anoj,6171,545,05/08/2025,Yes,Yes,10


In [71]:
output_file = 'tank_output.csv'
df.to_csv(output_file, index=False)
print(f'DataFrame saved to {output_file}')

DataFrame saved to tank_output.csv


In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [73]:
import pandas as pd


df = pd.read_csv("tank_output.csv")


df.head(5)


,Check all,Rack,Position,Tank ID,Label,% juvenile age,% adult age,# Total,Age level,Line / Strain (Name),Parents IDs,Owner,Children IDs,Animal age (d),Last crossing,Children Tank,Children Adults?,# children adults
0,Item 6543,NaN,NaN,5570/6543,NaN,76.0,76.0,2,Adult,mitfa -/-,4970,Satou Chie,"6632, 6677, 6678, 6707",376,02/21/2025,Yes,No,0
1,Item 6542,NaN,NaN,5570/6542,NaN,76.0,76.0,4,Adult,mitfa -/-,4970,Satou Chie,"6688, 6700, 6714, 6748",376,02/24/2025,Yes,No,0
2,Item 6497,NaN,NaN,5569/6497,NaN,74.0,74.0,2,Adult,mitfa -/-,4970,Satou Chie,"6679, 6706, 6815, 6824",377,03/26/2025,Yes,No,0
3,Item 6757,NaN,NaN,6387/6757,NaN,84.0,84.0,3,Adult,mitfa -/-,5417,Satou Chie,6853,154,04/05/2025,Yes,No,0
4,Item 5131,M-F09 Location Building: LSBArea: Aquatics 2...,A2,4555/5131,CID 12118-1,100.0,75.0,1,Adult,Tg(PB4:tTA;cryaa:mRuby); Tg(TRE:TEMPO-gRNA1),"3377, 3148",Ilanges Anoj,"5115, 5470, 5471",659,01/22/2024,Yes,Yes,15


In [74]:
# Filter the DataFrame for animals with age greater than 550 days
filtered_df = df[df['Animal age (d)'] > 550]

# Display the relevant columns (removed duplicate 'Owner')
result = filtered_df[['Tank ID', 'Line / Strain (Name)', 'Owner', 'Animal age (d)', 'Last crossing', 'Children Tank', 'Children Adults?', '# children adults']]

# Reset index and add a new ID column starting from 1
result = result.reset_index(drop=True)
result.index = result.index + 1
result.index.name = ''

# Display the first 5 rows
result.head(5)

,Tank ID,Line / Strain (Name),Owner,Animal age (d),Last crossing,Children Tank,Children Adults?,# children adults
,,,,,,,,
1,4555/5131,Tg(PB4:tTA;cryaa:mRuby); Tg(TRE:TEMPO-gRNA1),Ilanges Anoj,659,01/22/2024,Yes,Yes,15
2,4409/5299,Tg(her4.1:PMCA2-mCherry_J05),Ahrens Misha,713,02/25/2024,Yes,No,0
3,4598,Tg(sox10:CreERT2; cryaa:GFP); Tg(ubi:|s|-tTA; ...,Espinosa Medina Isabel,645,NaN,No,No,0
4,4601,Tg(sox10:CreERT2; cryaa:GFP); Tg(ubi:|s|-tTA; ...,Espinosa Medina Isabel,644,NaN,No,No,0
5,4601/4657,Tg(sox10:CreERT2; cryaa:GFP); Tg(ubi:|s|-tTA; ...,Espinosa Medina Isabel,644,NaN,No,No,0


In [75]:
result.to_excel('result_tanks.xlsx', index=False)

In [76]:
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys

In [77]:
def send_email_alert(subject, body, to_email):
    # sender and receiver
    sender_email = "yans2@hhmi.org"
    password = "Jtwmy&Dtsgx2025"
    receiver_email = "yansuyue320826@gmail.com"

    # SMTP server configuration
    smtp_server = "smtp.office365.com"
    smtp_port = 587

    # Email format
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Email content
    #msg.attach(MIMEText(body, 'plain'))

    try:
        # Connect to the SMTP server
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Secure the connection
        server.login(sender_email, password)

        # Send the email
        #server.sendmail(sender_email, receiver_email, msg.as_string())
        html = """\
        <html>
          <head></head>
          <body>
          <p>The following tanks are old.</p>
          {0}
        </body>
      </html>
      """.format(body.to_html())

        part1 = MIMEText(html, 'html')
        msg.attach(part1)

        #server = smtplib.SMTP('smtp.gmail.com', 587)
        server.sendmail(msg['From'], msg['To'] , msg.as_string())
        print("Email sent successfully!")

    except Exception as e:
        print(f"Failed to send email: {e}")

    finally:
        # Close the connection
        server.quit()

# Usage
subject = "Zebrafish Tank Alert: Old Tanks"
#body = (f"The following zebrafish tanks are old:df({result})")
body = result
receiver_email = "yansuyue320826@gmail.com"

send_email_alert(subject, body, receiver_email)

Email sent successfully!
